# Word2Vector 学习笔记

## 目录
1. [Word2Vector 简介](#1-word2vector-简介)
2. [核心思想](#2-核心思想)
3. [数学原理](#3-数学原理)
4. [Skip-gram 模型详解](#4-skip-gram-模型详解)
5. [CBOW 模型详解](#5-cbow-模型详解)
6. [Skip-gram vs CBOW 对比](#6-skip-gram-vs-cbow-对比)
7. [代码实现](#7-代码实现)
8. [实际应用示例](#8-实际应用示例)
9. [进阶技术](#9-进阶技术)
10. [练习题](#10-练习题)

## 1. Word2Vector 简介

Word2Vector是由Google在2013年提出的一种词嵌入技术，能够将词汇映射到高维向量空间中。它的核心思想是：**语义相似的词在向量空间中应该距离较近**。

### 主要特点：
- **分布式表示**：每个词用稠密的实数向量表示
- **语义相似性**：相似词的向量在空间中距离较近
- **计算效率**：相比传统方法，训练速度更快
- **向量运算**：支持词汇间的向量运算（如：king - man + woman ≈ queen）

## 2. 核心思想

Word2Vector基于**分布假设**（Distributional Hypothesis）：
> "You shall know a word by the company it keeps" - J.R. Firth

即：**一个词的含义由其上下文决定**。

### 基本原理：
1. 通过大量文本数据学习词汇的分布式表示
2. 利用神经网络预测词汇的上下文关系
3. 在训练过程中，语义相似的词会获得相似的向量表示

## 3. 数学原理

### 目标函数
Word2Vector的目标是最大化以下似然函数：

$$\mathcal{L} = \prod_{t=1}^{T} \prod_{-c \leq j \leq c, j \neq 0} p(w_{t+j} | w_t)$$

对数似然函数：
$$\log \mathcal{L} = \sum_{t=1}^{T} \sum_{-c \leq j \leq c, j \neq 0} \log p(w_{t+j} | w_t)$$

其中：
- $T$ 是语料库中词的总数
- $c$ 是上下文窗口大小
- $w_t$ 是第 $t$ 个词
- $w_{t+j}$ 是 $w_t$ 的上下文词

### Softmax 函数
条件概率通过softmax函数计算：

$$p(w_O | w_I) = \frac{\exp(v_{w_O}^T v_{w_I})}{\sum_{w=1}^{W} \exp(v_w^T v_{w_I})}$$

其中：
- $v_w$ 和 $v_w'$ 分别是词 $w$ 的输入和输出向量表示
- $W$ 是词汇表大小

## 4. Skip-gram 模型详解

### 4.1 基本思想
Skip-gram模型的核心思想是：**给定中心词，预测其上下文词**。

### 4.2 模型架构
```
输入层 → 隐藏层 → 输出层
  ↓        ↓        ↓
中心词 → 词向量 → 上下文词概率分布
```

### 4.3 数学推导

#### 目标函数
对于Skip-gram，我们要最大化：
$$\mathcal{J} = \frac{1}{T} \sum_{t=1}^{T} \sum_{-m \leq j \leq m, j \neq 0} \log p(w_{t+j} | w_t)$$

#### 条件概率
$$p(w_{t+j} | w_t) = \frac{\exp(u_{w_{t+j}}^T v_{w_t})}{\sum_{w=1}^{W} \exp(u_w^T v_{w_t})}$$

其中：
- $v_{w_t}$ 是中心词 $w_t$ 的输入向量
- $u_{w_{t+j}}$ 是上下文词 $w_{t+j}$ 的输出向量

#### 梯度计算
对于中心词向量的梯度：
$$\frac{\partial \log p(w_o | w_c)}{\partial v_c} = u_o - \sum_{w=1}^{W} p(w|w_c) u_w$$

### 4.4 Skip-gram 特点
- **适合大语料库**：在大数据集上表现更好
- **对低频词友好**：能更好地学习罕见词的表示
- **计算复杂度高**：需要预测多个上下文词

## 5. CBOW 模型详解

### 5.1 基本思想
CBOW（Continuous Bag of Words）模型的核心思想是：**给定上下文词，预测中心词**。

### 5.2 模型架构
```
输入层 → 隐藏层 → 输出层
  ↓        ↓        ↓
上下文词 → 平均词向量 → 中心词概率分布
```

### 5.3 数学推导

#### 目标函数
对于CBOW，我们要最大化：
$$\mathcal{J} = \frac{1}{T} \sum_{t=1}^{T} \log p(w_t | w_{t-m}, ..., w_{t-1}, w_{t+1}, ..., w_{t+m})$$

#### 上下文表示
CBOW将上下文词向量平均作为输入：
$$\hat{v} = \frac{1}{2m} \sum_{j=-m, j \neq 0}^{m} v_{w_{t+j}}$$

#### 条件概率
$$p(w_t | Context) = \frac{\exp(u_{w_t}^T \hat{v})}{\sum_{w=1}^{W} \exp(u_w^T \hat{v})}$$

#### 梯度计算
对于上下文词向量的梯度：
$$\frac{\partial \log p(w_c | Context)}{\partial v_{w_j}} = \frac{1}{2m} \left( u_{w_c} - \sum_{w=1}^{W} p(w|Context) u_w \right)$$

### 5.4 CBOW 特点
- **训练速度快**：只需预测一个中心词
- **适合小语料库**：在小数据集上表现更稳定
- **对高频词友好**：更好地学习常见词的表示
- **平滑效果**：通过平均上下文向量，减少噪声影响

## 6. Skip-gram vs CBOW 对比

| 特征 | Skip-gram | CBOW |
|------|-----------|------|
| **预测方向** | 中心词 → 上下文词 | 上下文词 → 中心词 |
| **训练速度** | 较慢 | 较快 |
| **数据集大小** | 适合大数据集 | 适合小数据集 |
| **低频词处理** | 表现更好 | 表现一般 |
| **高频词处理** | 表现一般 | 表现更好 |
| **语法信息** | 捕获较少 | 捕获较多 |
| **语义信息** | 捕获较多 | 捕获较少 |
| **计算复杂度** | O(C × log W) | O(log W) |

### 选择建议：
- **大语料库 + 关注语义**：选择 Skip-gram
- **小语料库 + 关注语法**：选择 CBOW
- **计算资源有限**：选择 CBOW
- **需要处理罕见词**：选择 Skip-gram

## 7. 代码实现

### 7.1 导入必要的库

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict, Counter
import re
from sklearn.decomposition import PCA
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

### 7.2 数据预处理类

In [ ]:
class Word2VecDataProcessor:
    def __init__(self, min_count=1, window_size=2):
        self.min_count = min_count
        self.window_size = window_size
        self.word2idx = {}
        self.idx2word = {}
        self.word_counts = Counter()
        self.vocab_size = 0
    
    def build_vocab(self, sentences):
        """构建词汇表"""
        # 统计词频
        for sentence in sentences:
            words = sentence.lower().split()
            self.word_counts.update(words)
        
        # 过滤低频词
        vocab = [word for word, count in self.word_counts.items() if count >= self.min_count]
        
        # 构建词汇映射
        self.word2idx = {word: idx for idx, word in enumerate(vocab)}
        self.idx2word = {idx: word for word, idx in self.word2idx.items()}
        self.vocab_size = len(vocab)
        
        print(f"词汇表大小: {self.vocab_size}")
    
    def generate_skipgram_data(self, sentences):
        """生成Skip-gram训练数据"""
        data = []
        for sentence in sentences:
            words = sentence.lower().split()
            for i, center_word in enumerate(words):
                if center_word not in self.word2idx:
                    continue
                
                center_idx = self.word2idx[center_word]
                
                # 获取上下文词
                for j in range(max(0, i - self.window_size), 
                             min(len(words), i + self.window_size + 1)):
                    if i != j and words[j] in self.word2idx:
                        context_idx = self.word2idx[words[j]]
                        data.append((center_idx, context_idx))
        
        return data
    
    def generate_cbow_data(self, sentences):
        """生成CBOW训练数据"""
        data = []
        for sentence in sentences:
            words = sentence.lower().split()
            for i, center_word in enumerate(words):
                if center_word not in self.word2idx:
                    continue
                
                center_idx = self.word2idx[center_word]
                context_indices = []
                
                # 获取上下文词
                for j in range(max(0, i - self.window_size), 
                             min(len(words), i + self.window_size + 1)):
                    if i != j and words[j] in self.word2idx:
                        context_indices.append(self.word2idx[words[j]])
                
                if context_indices:
                    data.append((context_indices, center_idx))
        
        return data

### 7.3 Skip-gram 模型实现

In [ ]:
class SkipGramModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(SkipGramModel, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        
        # 输入词嵌入层（中心词）
        self.in_embeddings = nn.Embedding(vocab_size, embedding_dim)
        # 输出词嵌入层（上下文词）
        self.out_embeddings = nn.Embedding(vocab_size, embedding_dim)
        
        # 初始化权重
        self.init_weights()
    
    def init_weights(self):
        """初始化权重"""
        initrange = 0.5 / self.embedding_dim
        self.in_embeddings.weight.data.uniform_(-initrange, initrange)
        self.out_embeddings.weight.data.uniform_(-initrange, initrange)
    
    def forward(self, center_word, context_word):
        """前向传播"""
        # 获取中心词向量
        center_embed = self.in_embeddings(center_word)  # [batch_size, embedding_dim]
        # 获取上下文词向量
        context_embed = self.out_embeddings(context_word)  # [batch_size, embedding_dim]
        
        # 计算点积
        score = torch.sum(center_embed * context_embed, dim=1)  # [batch_size]
        
        return score
    
    def get_embeddings(self):
        """获取词向量"""
        return self.in_embeddings.weight.data.cpu().numpy()

### 7.4 CBOW 模型实现

In [ ]:
class CBOWModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CBOWModel, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        
        # 输入词嵌入层（上下文词）
        self.in_embeddings = nn.Embedding(vocab_size, embedding_dim)
        # 输出词嵌入层（中心词）
        self.out_embeddings = nn.Embedding(vocab_size, embedding_dim)
        
        # 初始化权重
        self.init_weights()
    
    def init_weights(self):
        """初始化权重"""
        initrange = 0.5 / self.embedding_dim
        self.in_embeddings.weight.data.uniform_(-initrange, initrange)
        self.out_embeddings.weight.data.uniform_(-initrange, initrange)
    
    def forward(self, context_words, center_word):
        """前向传播"""
        # 获取上下文词向量并平均
        context_embeds = self.in_embeddings(context_words)  # [batch_size, context_size, embedding_dim]
        context_mean = torch.mean(context_embeds, dim=1)    # [batch_size, embedding_dim]
        
        # 获取中心词向量
        center_embed = self.out_embeddings(center_word)     # [batch_size, embedding_dim]
        
        # 计算点积
        score = torch.sum(context_mean * center_embed, dim=1)  # [batch_size]
        
        return score
    
    def get_embeddings(self):
        """获取词向量"""
        return self.in_embeddings.weight.data.cpu().numpy()

### 7.5 训练函数

In [ ]:
def train_skipgram(model, data, processor, epochs=100, lr=0.01):
    """训练Skip-gram模型"""
    optimizer = optim.SGD(model.parameters(), lr=lr)
    criterion = nn.BCEWithLogitsLoss()
    
    losses = []
    
    for epoch in range(epochs):
        total_loss = 0
        
        for center_idx, context_idx in data:
            # 正样本
            center_tensor = torch.LongTensor([center_idx])
            context_tensor = torch.LongTensor([context_idx])
            
            # 负采样
            neg_samples = np.random.choice(processor.vocab_size, 5, replace=False)
            neg_samples = [idx for idx in neg_samples if idx != context_idx]
            
            # 计算正样本损失
            pos_score = model(center_tensor, context_tensor)
            pos_loss = criterion(pos_score, torch.ones_like(pos_score))
            
            # 计算负样本损失
            neg_loss = 0
            for neg_idx in neg_samples[:4]:  # 使用4个负样本
                neg_tensor = torch.LongTensor([neg_idx])
                neg_score = model(center_tensor, neg_tensor)
                neg_loss += criterion(neg_score, torch.zeros_like(neg_score))
            
            loss = pos_loss + neg_loss
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        avg_loss = total_loss / len(data)
        losses.append(avg_loss)
        
        if epoch % 20 == 0:
            print(f"Epoch {epoch}, Loss: {avg_loss:.4f}")
    
    return losses

def train_cbow(model, data, processor, epochs=100, lr=0.01):
    """训练CBOW模型"""
    optimizer = optim.SGD(model.parameters(), lr=lr)
    criterion = nn.BCEWithLogitsLoss()
    
    losses = []
    
    for epoch in range(epochs):
        total_loss = 0
        
        for context_indices, center_idx in data:
            if len(context_indices) == 0:
                continue
                
            # 填充上下文词到固定长度
            max_context_len = 4
            if len(context_indices) < max_context_len:
                context_indices.extend([0] * (max_context_len - len(context_indices)))
            else:
                context_indices = context_indices[:max_context_len]
            
            context_tensor = torch.LongTensor([context_indices])
            center_tensor = torch.LongTensor([center_idx])
            
            # 负采样
            neg_samples = np.random.choice(processor.vocab_size, 5, replace=False)
            neg_samples = [idx for idx in neg_samples if idx != center_idx]
            
            # 计算正样本损失
            pos_score = model(context_tensor, center_tensor)
            pos_loss = criterion(pos_score, torch.ones_like(pos_score))
            
            # 计算负样本损失
            neg_loss = 0
            for neg_idx in neg_samples[:4]:  # 使用4个负样本
                neg_tensor = torch.LongTensor([neg_idx])
                neg_score = model(context_tensor, neg_tensor)
                neg_loss += criterion(neg_score, torch.zeros_like(neg_score))
            
            loss = pos_loss + neg_loss
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        avg_loss = total_loss / len(data)
        losses.append(avg_loss)
        
        if epoch % 20 == 0:
            print(f"Epoch {epoch}, Loss: {avg_loss:.4f}")
    
    return losses

## 8. 实际应用示例

### 8.1 准备示例数据

In [ ]:
# 示例语料
sentences = [
    "the quick brown fox jumps over the lazy dog",
    "a quick brown dog runs fast",
    "the lazy cat sleeps all day",
    "brown animals are beautiful",
    "the dog and cat are friends",
    "quick animals run fast",
    "the fox is clever and quick",
    "lazy animals sleep a lot",
    "brown fox and brown dog",
    "beautiful animals are quick"
]

print("示例语料:")
for i, sentence in enumerate(sentences):
    print(f"{i+1}. {sentence}")

### 8.2 训练 Skip-gram 模型

In [ ]:
# 初始化数据处理器
processor_sg = Word2VecDataProcessor(min_count=1, window_size=2)
processor_sg.build_vocab(sentences)

# 生成训练数据
skipgram_data = processor_sg.generate_skipgram_data(sentences)
print(f"Skip-gram训练样本数: {len(skipgram_data)}")

# 创建模型
skipgram_model = SkipGramModel(processor_sg.vocab_size, embedding_dim=50)

# 训练模型
print("\n开始训练Skip-gram模型...")
sg_losses = train_skipgram(skipgram_model, skipgram_data, processor_sg, epochs=100, lr=0.1)

### 8.3 训练 CBOW 模型

In [ ]:
# 初始化数据处理器
processor_cbow = Word2VecDataProcessor(min_count=1, window_size=2)
processor_cbow.build_vocab(sentences)

# 生成训练数据
cbow_data = processor_cbow.generate_cbow_data(sentences)
print(f"CBOW训练样本数: {len(cbow_data)}")

# 创建模型
cbow_model = CBOWModel(processor_cbow.vocab_size, embedding_dim=50)

# 训练模型
print("\n开始训练CBOW模型...")
cbow_losses = train_cbow(cbow_model, cbow_data, processor_cbow, epochs=100, lr=0.1)

### 8.4 可视化训练过程

In [ ]:
# 绘制损失曲线
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(sg_losses, label='Skip-gram', color='blue')
plt.title('Skip-gram 训练损失')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(cbow_losses, label='CBOW', color='red')
plt.title('CBOW 训练损失')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# 对比两个模型的损失
plt.figure(figsize=(10, 6))
plt.plot(sg_losses, label='Skip-gram', color='blue', linewidth=2)
plt.plot(cbow_losses, label='CBOW', color='red', linewidth=2)
plt.title('Skip-gram vs CBOW 训练损失对比')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

### 8.5 词向量可视化

In [ ]:
def visualize_embeddings(embeddings, word2idx, idx2word, title):
    """可视化词向量"""
    # 使用PCA降维到2D
    pca = PCA(n_components=2)
    embeddings_2d = pca.fit_transform(embeddings)
    
    plt.figure(figsize=(12, 8))
    
    # 绘制词向量点
    plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], alpha=0.7, s=100)
    
    # 添加词汇标签
    for i, word in idx2word.items():
        plt.annotate(word, (embeddings_2d[i, 0], embeddings_2d[i, 1]), 
                    xytext=(5, 5), textcoords='offset points', 
                    fontsize=12, alpha=0.8)
    
    plt.title(f'{title} 词向量可视化 (PCA降维)', fontsize=16)
    plt.xlabel('第一主成分')
    plt.ylabel('第二主成分')
    plt.grid(True, alpha=0.3)
    plt.show()

# 获取词向量
sg_embeddings = skipgram_model.get_embeddings()
cbow_embeddings = cbow_model.get_embeddings()

# 可视化Skip-gram词向量
visualize_embeddings(sg_embeddings, processor_sg.word2idx, processor_sg.idx2word, "Skip-gram")

# 可视化CBOW词向量
visualize_embeddings(cbow_embeddings, processor_cbow.word2idx, processor_cbow.idx2word, "CBOW")

### 8.6 词汇相似度分析

In [ ]:
def cosine_similarity(vec1, vec2):
    """计算余弦相似度"""
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    return dot_product / (norm1 * norm2)

def find_similar_words(word, embeddings, word2idx, idx2word, top_k=5):
    """找到最相似的词"""
    if word not in word2idx:
        return []
    
    word_idx = word2idx[word]
    word_vec = embeddings[word_idx]
    
    similarities = []
    for idx, other_word in idx2word.items():
        if idx != word_idx:
            other_vec = embeddings[idx]
            sim = cosine_similarity(word_vec, other_vec)
            similarities.append((other_word, sim))
    
    # 按相似度排序
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:top_k]

# 测试词汇相似度
test_words = ['quick', 'brown', 'dog', 'lazy']

print("=== Skip-gram 词汇相似度 ===")
for word in test_words:
    similar_words = find_similar_words(word, sg_embeddings, processor_sg.word2idx, processor_sg.idx2word)
    print(f"\n'{word}' 的相似词:")
    for similar_word, similarity in similar_words:
        print(f"  {similar_word}: {similarity:.4f}")

print("\n=== CBOW 词汇相似度 ===")
for word in test_words:
    similar_words = find_similar_words(word, cbow_embeddings, processor_cbow.word2idx, processor_cbow.idx2word)
    print(f"\n'{word}' 的相似词:")
    for similar_word, similarity in similar_words:
        print(f"  {similar_word}: {similarity:.4f}")

## 9. 进阶技术

### 9.1 负采样 (Negative Sampling)

负采样是Word2Vector中的一个重要优化技术，用于解决softmax计算复杂度过高的问题。

#### 基本思想：
- 不计算所有词的softmax概率
- 只对少数负样本进行计算
- 将多分类问题转化为二分类问题

#### 数学公式：
$$\log \sigma(v_{w_O}^T v_{w_I}) + \sum_{i=1}^{k} \mathbb{E}_{w_i \sim P_n(w)} [\log \sigma(-v_{w_i}^T v_{w_I})]$$

其中：
- $\sigma(x) = \frac{1}{1 + e^{-x}}$ 是sigmoid函数
- $k$ 是负样本数量
- $P_n(w)$ 是负采样分布

#### 负采样分布：
$$P(w_i) = \frac{f(w_i)^{3/4}}{\sum_{j=0}^{n} f(w_j)^{3/4}}$$

### 9.2 层次化Softmax (Hierarchical Softmax)

层次化Softmax使用二叉树结构来减少计算复杂度。

#### 特点：
- 将词汇表组织成二叉树
- 每个叶子节点代表一个词
- 计算复杂度从 $O(V)$ 降到 $O(\log V)$

### 9.3 子词信息 (Subword Information)

FastText扩展了Word2Vector，加入了子词信息。

#### 优势：
- 处理未登录词（OOV）
- 捕获词汇的形态学信息
- 对拼写错误更鲁棒

## 10. 练习题

### 理论题

1. **基础概念**
   - 解释Word2Vector的核心思想
   - 什么是分布假设？
   - Skip-gram和CBOW的主要区别是什么？

2. **数学推导**
   - 推导Skip-gram的目标函数
   - 解释为什么需要负采样
   - 计算softmax函数的梯度

3. **模型对比**
   - 在什么情况下选择Skip-gram？
   - 在什么情况下选择CBOW？
   - 比较两种模型的计算复杂度

### 实践题

1. **代码实现**
   - 实现一个简单的负采样函数
   - 修改训练函数，加入学习率衰减
   - 实现词汇相似度搜索功能

2. **实验设计**
   - 比较不同窗口大小对模型性能的影响
   - 测试不同嵌入维度的效果
   - 分析负采样数量对训练效果的影响

3. **应用拓展**
   - 使用预训练的Word2Vector模型
   - 实现词汇类比任务（如：king - man + woman = queen）
   - 将Word2Vector应用到文本分类任务中

### 思考题

1. Word2Vector有哪些局限性？
2. 如何处理多义词问题？
3. Word2Vector与现代Transformer模型的关系？
4. 如何评估词向量的质量？

---

## 总结

Word2Vector是自然语言处理领域的重要突破，它开启了词嵌入的新时代。通过本笔记，我们深入学习了：

1. **理论基础**：分布假设、目标函数、数学推导
2. **模型架构**：Skip-gram和CBOW的详细对比
3. **实现细节**：从数据预处理到模型训练的完整流程
4. **优化技术**：负采样、层次化Softmax等进阶方法
5. **实际应用**：词汇相似度、可视化、类比推理

Word2Vector为后续的深度学习模型奠定了基础，理解其原理对于掌握现代NLP技术具有重要意义。